In [27]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv('unprocessed_lyrics.csv')
df = df.groupby('Genre', group_keys=False).apply(lambda s: s.sample(1000, random_state=42)) #


In [28]:
df

,SName,Lyric,Genre,Artist,lyric_length
2157,What You Want,"[Intro]. Ladies and gentlemen. We are gathered here today. To join. I see it, yeah. Yeah, yeah. I like this right here. Yeah, yeah. One-Two, One-Two, in the place to be. As you can see. I am the c...",Hip Hop,Puff Daddy,572
3187,Don't Throw Us Away,In a perfect world. I would still be your girl. In a perfect dream. You would still be with me. But nothing's perfect cus I'm still hurtin. Here I am all alone with a broken heart. Heart. I don't ...,Hip Hop,Ester Dean,365
9164,Middle Child,"You good, T-Minus?. Niggas been countin' me out. I'm countin' my bullets, I'm loadin' my clips. I'm writin' down names, I'm makin' a list. I'm checkin' it twice and I'm gettin' 'em hit. The real o...",Hip Hop,J. Cole,719
2145,Leave Me Alone,"[Snoop]. Yo.. uh-huh. I'ma try somethin different on this right here. but I know y'all gon' feel it. cause I know y'all feel me. It's some real shit right here, y'knahmsayin?. I ain't never did it...",Hip Hop,Snoop Dogg,557
6442,Just Lose It,"{*Eminem making sounds*}. Okay. Guess who's back, back again. Shady's back, tell a friend. Now everyone report to the dance floor. To the dance floor, to the dance floor. Now everyone report to th...",Hip Hop,Eminem,725
...,...,...,...,...,...
36875,Send Away The Tigers,There’s no hope in the colonies. So catch yourself a lifeline. Things have gone wrong too many times. So catch yourself a slow boat to china. Fixing some holes from the tigers claws. All your tear...,Rock,Manic Street Preachers,184
38526,Knights Of Cydonia,Come ride with me. Through the veins of history. I'll show you a god. Who falls asleep on the job. And how can we win. When fools can be kings. Don't waste your time. Or time will waste you. No on...,Rock,Muse,126
32623,What If It All Means Something,If I could do anything for you. Believe me I would. Do you feel the same?. Feel the same as me?. If I could be anyone for you. Believe me I would. I?m not ashamed. Not ashamed to be. It?s hard for...,Rock,Chantal Kreviazuk,316
36551,Love Like a Sunset Part I,Acres. Visible horizon. Right where it starts and ends. Oh and then we start the end. Acres. A visible illusion. Oh where it starts it ends. You are like a sunset,Rock,Phoenix,31


In [29]:
# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
assert 'GPU' in str(device_lib.list_local_devices())

# confirm Keras sees the GPU (for TensorFlow 1.X + Keras)
from keras import backend
assert len(backend.tensorflow_backend._get_available_gpus()) > 0

In [30]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) # list of DeviceAttributes



[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15406044623610518844
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8329998636343240033
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 674808946735445687
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14949928141
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15479246909865147292
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:1e.0, compute capability: 7.5"
]


In [31]:

df["label"] = LabelEncoder().fit_transform(df["Genre"])


In [32]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)

In [33]:
#!pip install tensorflow_hub
import tensorflow_hub as hub


embed = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [34]:
import tensorflow as tf
import pandas as pd
import keras

from sklearn import preprocessing
from keras.utils import np_utils
import numpy as np


y = list(df['label'])
x = list(df['Lyric'])

le = preprocessing.LabelEncoder()
le.fit(y)

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.np_utils.to_categorical(enc)

def decode(le, one_hot):
    print(one_hot)
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)


x_enc = x
y_enc = encode(le, y)

In [35]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(np.asarray(x_enc), np.asarray(y_enc), test_size=0.2, random_state=42)

In [36]:
x_train.shape


(2400,)

In [37]:
from keras.layers import Input, Lambda, Dense
import tensorflow as tf

input_text = Input(shape=(1,), dtype=tf.string)
print(input_text)

Tensor("input_6:0", shape=(?, 1), dtype=string)


In [38]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

input_text = Input(shape=(1,), dtype=tf.string)

embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text) # elmo vectors are of size 1024. Our first layer (input layer)
dense = Dense(256, activation='relu')(embedding) # second layer, (dense layer) each node is connected to all nodes of the preceding layer
pred = Dense(3, activation='softmax')(dense) # output layer
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=3, batch_size=16)
    predicts = model.predict(x_test, batch_size=16)
    #model.save_weights('./elmo-model.h5')

y_test = decode(le, y_test)
y_preds = decode(le, predicts)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:1e.0, compute capability: 7.5

Epoch 1/3
2400/2400 [==============================] - 436s 182ms/step - loss: 0.8575 - acc: 0.6042
Epoch 2/3
2400/2400 [==============================] - 370s 154ms/step - loss: 0.7558 - acc: 0.6362
Epoch 3/3
2400/2400 [==============================] - 352s 147ms/step - loss: 0.7386 - acc: 0.6583
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]
[[1.2161297e-01 5.0545454e-01 3.7293255e-01]
 [4.6397731e-01 4.6439740e-01 7.1625315e-02]
 [8.9550269e-01 8.8913463e-02 1.5583757e-02]
 ...
 [9.9615085e-01 3.3364482e-03 5.1275908e-04]
 [6.1766517e-01 2.8925583e-01 9.3078934e-02]
 [2.1451554e-01 4.2613980e-01 3.5934466e-01]]


In [39]:
from sklearn import metrics

print(metrics.confusion_matrix(y_test, y_preds))

print(metrics.classification_report(y_test, y_preds))

from sklearn.metrics import accuracy_score

print("Accuracy of ELMO is:",accuracy_score(y_test,y_preds))

[[180  26  11]
 [ 60  94  43]
 [ 21  68  97]]
              precision    recall  f1-score   support

           0       0.69      0.83      0.75       217
           1       0.50      0.48      0.49       197
           2       0.64      0.52      0.58       186

    accuracy                           0.62       600
   macro avg       0.61      0.61      0.61       600
weighted avg       0.61      0.62      0.61       600

Accuracy of ELMO is: 0.6183333333333333
